<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_9_RAG_API.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage RAG API (experimental)



## Environment

In [1]:
!pip install -qU langchain-upstage langchain python-dotenv openai

### Environment variables

Set up environment variables 
* UPSTAGE_API_KEY


In [2]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
rag_base_url = "https://experimental-api.x.upstage.ai/rag-api/"

# Plase chose your model name
rag_model_name = "my_test_model"

In [4]:
from openai import OpenAI


client = OpenAI(
    base_url=rag_base_url,
    api_key=os.environ["UPSTAGE_API_KEY"]
)

print("Building/Rebuilding the index...")
# Upload all files in the pdfs folder
for file in os.listdir("pdfs"):
    file_path = os.path.join("pdfs", file)
    with open(file_path, "rb") as f:
        file_response = client.files.create(
            file=f,
            purpose="assistants",
            extra_body={"model_name": rag_model_name}
        )
        pprint(file_response)

print("Index build/rebuild submission completed.")

public_model_name = file_response.public_model_name
print("Public Mode Name:", public_model_name)

Building/Rebuilding the index...
FileObject(id='file-13', bytes=418872, created_at=1727452439, filename='solar_paper.pdf', object='file', purpose='assistants', status='indexing', status_details=None, model_name='my_test_model', public_model_name='788a5ea436f3968df73b', message='File solar_paper.pdf is already indexed, skipping indexing')
FileObject(id='file-14', bytes=137425, created_at=1727452440, filename='solar_sample.pdf', object='file', purpose='assistants', status='indexing', status_details=None, model_name='my_test_model', public_model_name='788a5ea436f3968df73b', message='File solar_sample.pdf is already indexed, skipping indexing')
FileObject(id='file-15', bytes=878624, created_at=1727452442, filename='docai.pdf', object='file', purpose='assistants', status='indexing', status_details=None, model_name='my_test_model', public_model_name='788a5ea436f3968df73b', message='File docai.pdf is already indexed, skipping indexing')
FileObject(id='file-16', bytes=4432693, created_at=17274

In [5]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ["UPSTAGE_API_KEY"]  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[
        {"role": "user", "content": "Explain Document AI"}  # The user's query
    ],
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_query": True,  # Enable contextual querying
        "verbose": True  # Request verbose output for debugging
    }
)

# Print the model's response
pprint(response.choices[0].message)

# Print verbose information (if available)
# This loop iterates through additional usage information provided by the API
for info in response.usage:
    pprint(info.page_content)

ChatCompletionMessage(content='Upstage Document AI is a product that includes three solutions: Layout Analyzer, Key Information Extractor, and Document OCR. These solutions can extract layouts, tables, figures, key information, and all text from any document. In various enterprise use cases, Upstage Document AI has shown higher accuracy (95%-98% accuracy rate) compared to previous manual human work for document processing.', refusal=None, role='assistant', function_call=None, tool_calls=None)
('Document AI: Digitize anythingUpstage Document AI is comprised of three '
 'solutions:● Layout Analyzer: Extracts layouts, tables, and figures from any '
 'document.\n'
 '● Key Information Extractor: Extracts key information from target '
 'documents.\n'
 '● Document OCR: Extracts all text from any document.In various enterprise '
 'use cases, Upstage Document AI resulted in higher accuracy\n'
 '(95%-98% accuracy rate) compared to previous manual human work for document')
('(95%-98% accuracy rat

In [6]:
# Import necessary libraries
from pprint import pprint
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(model=rag_model_name, base_url=rag_base_url)

# Send a query to the chat model
question = "What's the Document AI?"
response = chat.invoke(question)

# Print the response content in a formatted way
pprint(response.content)

('Based on the context provided, Document AI is a product that includes three '
 'solutions: Layout Analyzer, Key Information Extractor, and Document OCR. '
 'These solutions can extract layouts, tables, figures, key information, and '
 'all text from any document. In various enterprise use cases, Document AI has '
 'shown higher accuracy (95%-98% accuracy rate) compared to previous manual '
 'human work for document processing.')


In [7]:
# Initialize the chat model using the public model name (accessible to everyone)
chat = Chat(model=public_model_name, base_url=rag_base_url)

# Define the question about Document AI and its usage
question = "What's Document AI? How can it be used?"

# Invoke the chat model with the question
response = chat.invoke(question)

# Print the question for context
print(f"Question: {question}")

# Pretty print the response content for better readability
print("Response:")
pprint(response.content)

Question: What's Document AI? How can it be used?
Response:
('Upstage Document AI is a product that includes three solutions: Layout '
 'Analyzer, Key Information Extractor, and Document OCR. These solutions can '
 'extract layouts, tables, figures, key information, and all text from any '
 'document. In various enterprise use cases, Upstage Document AI has shown '
 'higher accuracy (95%-98% accuracy rate) compared to previous manual human '
 'work for document processing. The solution has been heavily adopted by '
 'various leading insurance providers, yielding significant productivity gains '
 'and operating cost reductions. Additional information such as customer '
 'testimonials, key customers, and customer success stories are available.')
